In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('income').getOrCreate()

In [2]:
from pyspark.sql.types import *

# schema = StructType([
#     StructField("age", IntegerType(), True), 
#     StructField("workclass", StringType(), True),
#     StructField("fnlwgt", FloatType(), True),
#     StructField("education", StringType(), True),
#     StructField("education-num", FloatType(), True),
#     StructField("marital-status", StringType(), True),
#     StructField("occupation", StringType(), True),
#     StructField("relationship", StringType(), True),
#     StructField("race", StringType(), True),
#     StructField("sex", StringType(), True),
#     StructField("capital-gain", FloatType(), True),
#     StructField("capital-loss", FloatType(), True),
#     StructField("hours-per-week", FloatType(), True),
#     StructField("native-country", StringType(), True),
#     StructField("class", StringType(), True)]
# )

# train = spark.read.csv('./adult.data.txt', schema=schema, inferSchema='true')

headers = ["age", "workclass", "fnlwgt", "education", "education-num",
           "marital-status", "occupation", "relationship", "race", "sex",
           "capital-gain", "capital-loss", "hours-per-week", "native-country",
           "class"]

train = spark.read.csv('./adult.data.txt',
                       inferSchema='true', 
                       ignoreLeadingWhiteSpace='true',
                       ignoreTrailingWhiteSpace='true').toDF(*headers)
train.printSchema()

root
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: integer (nullable = true)
 |-- education: string (nullable = true)
 |-- education-num: integer (nullable = true)
 |-- marital-status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- capital-gain: integer (nullable = true)
 |-- capital-loss: integer (nullable = true)
 |-- hours-per-week: integer (nullable = true)
 |-- native-country: string (nullable = true)
 |-- class: string (nullable = true)



In [3]:
from pyspark.sql.functions import udf, monotonically_increasing_id

train = train.withColumn('index', monotonically_increasing_id())

In [4]:
labelCol = 'class'
train.count()

32561

# Exploratory Data Analysis

In [5]:
train.groupby(labelCol).count().show()

+-----+-----+
|class|count|
+-----+-----+
|<=50K|24720|
| >50K| 7841|
+-----+-----+



We can see there is a class imbalance problem in our training set.

In [6]:
from pyspark.sql.functions import col

def findMissingValuesCols(df):
    numRows = df.count()
    nullCols = []
    for column in df.columns:
        c = df.filter(col(column).isNotNull()).count()
        if c != numRows:
            nullCols.append(c)
    return nullCols

findMissingValuesCols(train)

[]

In [7]:
train.select('age').distinct().show()

+---+
|age|
+---+
| 31|
| 85|
| 65|
| 53|
| 78|
| 34|
| 81|
| 28|
| 76|
| 27|
| 26|
| 44|
| 22|
| 47|
| 52|
| 86|
| 40|
| 20|
| 57|
| 54|
+---+
only showing top 20 rows



In [8]:
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import format_number

def crosstabPercentage(df, col1, col2):
    ctabDf = df.crosstab(col1, col2)
    ctabCol = col1 + '_' + col2
    ctabNewCol = col1 + col2.title()
    ctabDf = ctabDf.withColumn(ctabNewCol, ctabDf[ctabCol])\
                                         .orderBy(ctabNewCol).drop(ctabCol)
    # Strip extra whitespaces from column name
    for column in ctabDf.columns:
        columnStripped = column.strip()
        if column != columnStripped:
            ctabDf = ctabDf.withColumn(column.strip(), ctabDf[column])\
                                         .drop(column)

    ctabDf = ctabDf.withColumn('percentage-of->50K', ctabDf['>50K']/(ctabDf['<=50K']+ctabDf['>50K'])*100)
    return ctabDf


In [9]:
df = crosstabPercentage(train, 'race', labelCol).orderBy('percentage-of->50K')
df = df.orderBy('percentage-of->50K').withColumn('percentage-of->50K', 
                                    format_number(df['percentage-of->50K'], 2))
df.show()

+-----+----+------------------+------------------+
|<=50K|>50K|         raceClass|percentage-of->50K|
+-----+----+------------------+------------------+
|  246|  25|             Other|              9.23|
|  275|  36|Amer-Indian-Eskimo|             11.58|
| 2737| 387|             Black|             12.39|
|20699|7117|             White|             25.59|
|  763| 276|Asian-Pac-Islander|             26.56|
+-----+----+------------------+------------------+



In [10]:
df = crosstabPercentage(train, 'age', labelCol).orderBy('percentage-of->50K')
df = df.orderBy('ageClass').withColumn('percentage-of->50K', 
                                    format_number(df['percentage-of->50K'], 2))
df.show(df.count())

+-----+----+--------+------------------+
|<=50K|>50K|ageClass|percentage-of->50K|
+-----+----+--------+------------------+
|  395|   0|      17|              0.00|
|  550|   0|      18|              0.00|
|  710|   2|      19|              0.28|
|  753|   0|      20|              0.00|
|  717|   3|      21|              0.42|
|  752|  13|      22|              1.70|
|  865|  12|      23|              1.37|
|  767|  31|      24|              3.88|
|  788|  53|      25|              6.30|
|  722|  63|      26|              8.03|
|  754|  81|      27|              9.70|
|  748| 119|      28|             13.73|
|  679| 134|      29|             16.48|
|  690| 171|      30|             19.86|
|  705| 183|      31|             20.61|
|  639| 189|      32|             22.83|
|  684| 191|      33|             21.83|
|  643| 243|      34|             27.43|
|  659| 217|      35|             24.77|
|  635| 263|      36|             29.29|
|  566| 292|      37|             34.03|
|  545| 282|    

In [11]:
df = crosstabPercentage(train, 'sex', labelCol)
df = df.orderBy('percentage-of->50K').withColumn('percentage-of->50K', 
                                    format_number(df['percentage-of->50K'], 2))
df.show()


+-----+----+--------+------------------+
|<=50K|>50K|sexClass|percentage-of->50K|
+-----+----+--------+------------------+
| 9592|1179|  Female|             10.95|
|15128|6662|    Male|             30.57|
+-----+----+--------+------------------+



In [12]:
df = crosstabPercentage(train, 'education', labelCol).orderBy('percentage-of->50K')
df = df.orderBy('percentage-of->50K').withColumn('percentage-of->50K', 
                                    format_number(df['percentage-of->50K'], 2))
df.show()


+-----+----+--------------+------------------+
|<=50K|>50K|educationClass|percentage-of->50K|
+-----+----+--------------+------------------+
|   51|   0|     Preschool|              0.00|
|  162|   6|       1st-4th|              3.57|
|  317|  16|       5th-6th|              4.80|
| 1115|  60|          11th|              5.11|
|  487|  27|           9th|              5.25|
|  606|  40|       7th-8th|              6.19|
|  871|  62|          10th|              6.65|
|  400|  33|          12th|              7.62|
| 8826|1675|       HS-grad|             15.95|
| 5904|1387|  Some-college|             19.02|
|  802| 265|    Assoc-acdm|             24.84|
| 1021| 361|     Assoc-voc|             26.12|
| 3134|2221|     Bachelors|             41.48|
|  764| 959|       Masters|             55.66|
|  153| 423|   Prof-school|             73.44|
|  107| 306|     Doctorate|             74.09|
+-----+----+--------------+------------------+



In [13]:
educationNumClass = crosstabPercentage(train, 'education-num', labelCol)
educationNumClass = educationNumClass.withColumn('percentage-of->50K', 
                                    format_number(educationNumClass['percentage-of->50K'], 2))
educationNumClass = educationNumClass.withColumn('education-numClassF', educationNumClass['education-numClass'].cast(DoubleType()))\
                                     .orderBy('education-numClassF').drop('education-numClass')
cols = educationNumClass.columns
cols.remove('education-numClassF')
cols.insert(0, 'education-numClassF')
educationNumClass = educationNumClass.select(cols)
educationNumClass.show()


+-------------------+-----+----+------------------+
|education-numClassF|<=50K|>50K|percentage-of->50K|
+-------------------+-----+----+------------------+
|                1.0|   51|   0|              0.00|
|                2.0|  162|   6|              3.57|
|                3.0|  317|  16|              4.80|
|                4.0|  606|  40|              6.19|
|                5.0|  487|  27|              5.25|
|                6.0|  871|  62|              6.65|
|                7.0| 1115|  60|              5.11|
|                8.0|  400|  33|              7.62|
|                9.0| 8826|1675|             15.95|
|               10.0| 5904|1387|             19.02|
|               11.0| 1021| 361|             26.12|
|               12.0|  802| 265|             24.84|
|               13.0| 3134|2221|             41.48|
|               14.0|  764| 959|             55.66|
|               15.0|  153| 423|             73.44|
|               16.0|  107| 306|             74.09|
+-----------

In [14]:
df = train.crosstab('education-num', 'education')
df.show()

+-----------------------+----+----+----+-------+-------+-------+---+----------+---------+---------+---------+-------+-------+---------+-----------+------------+
|education-num_education|10th|11th|12th|1st-4th|5th-6th|7th-8th|9th|Assoc-acdm|Assoc-voc|Bachelors|Doctorate|HS-grad|Masters|Preschool|Prof-school|Some-college|
+-----------------------+----+----+----+-------+-------+-------+---+----------+---------+---------+---------+-------+-------+---------+-----------+------------+
|                      5|   0|   0|   0|      0|      0|      0|514|         0|        0|        0|        0|      0|      0|        0|          0|           0|
|                     10|   0|   0|   0|      0|      0|      0|  0|         0|        0|        0|        0|      0|      0|        0|          0|        7291|
|                     14|   0|   0|   0|      0|      0|      0|  0|         0|        0|        0|        0|      0|   1723|        0|          0|           0|
|                      1|   0|   0

We can see above that this is a sparse matrix, it's hard to find the non-zero values. So we will only focus on non-zero values to find out whether there is any relationship between these features and one of them is redundant.

In [15]:
from pyspark.sql.functions import coalesce, lit, when

iterator = df.toLocalIterator()
d = {}
for row in iterator:
    rowDict = row.asDict()
    educationNum = rowDict['education-num_education']
    for k, v in rowDict.items():
        if k != 'education-num_education' and v != 0:
            d[educationNum+'_'+k] = v

import json
s = json.dumps(d, indent=4)
print(s)

{
    "5_9th": 514,
    "10_Some-college": 7291,
    "14_Masters": 1723,
    "1_Preschool": 51,
    "6_10th": 933,
    "9_HS-grad": 10501,
    "13_Bachelors": 5355,
    "2_1st-4th": 168,
    "12_Assoc-acdm": 1067,
    "7_11th": 1175,
    "3_5th-6th": 333,
    "16_Doctorate": 413,
    "11_Assoc-voc": 1382,
    "8_12th": 433,
    "4_7th-8th": 646,
    "15_Prof-school": 576
}


We can see it's obvious that these features are redundant. Only one of them should suffice for our classification task.

Let's try more rigorous chi square test instead of something hand-wavy.

First we will define an utility method that'll index the catgorical string columns, encodes them into one-hot-encoded vectors, and finally assemble all the feature vectos into once vector for later downstream analysis.

In [16]:
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml.stat import ChiSquareTest
from pyspark.ml import Pipeline

def autoIndexer(df, lableCol, outputCol='assembled'):
    stringTypes = [dtype[0] for dtype in df.dtypes if dtype[1] == 'string']
    indexedTypes = [stringType+'Indexed' for stringType in stringTypes]
    try:
        indexedTypes.remove(lableCol+'Indexed')
    except:
        pass
    indexers = [StringIndexer(inputCol=stringType, outputCol=stringType+'Indexed') for stringType in stringTypes]
    oheTypes = [indexedType+'OneHotEncoded' for indexedType in indexedTypes]
    ohe = OneHotEncoderEstimator(inputCols=indexedTypes, outputCols=oheTypes)
    assembler = VectorAssembler(inputCols=oheTypes, outputCol=outputCol)
    pipeline = Pipeline(stages=[*indexers, ohe, assembler])    
    indexed = pipeline.fit(df).transform(df)
    return stringTypes, oheTypes, indexed

In [17]:
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml.stat import ChiSquareTest
from pyspark.ml import Pipeline

indexed = train.select('education-num', 'education')

indexer = StringIndexer(inputCol='education', outputCol='educationIndexed')
indexed = indexer.fit(indexed).transform(indexed)
ohe = OneHotEncoderEstimator(inputCols=['education-num',], outputCols=['education-numOHE',])
indexed = ohe.fit(indexed).transform(indexed)

# The null hypothesis is that the occurrence of the outcomes is statistically independent.
# In general, small p-values (1% to 5%) would cause you to reject the null hypothesis. 
# This very large p-value (92.65%) means that the null hypothesis should not be rejected.
testResult = ChiSquareTest.test(indexed, 'education-numOHE', 'educationIndexed')
r = testResult.head()
print("pValues: " + str(r.pValues))

pValues: [1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]


We can accept the hypothesis that features are dependent. We will drop the 'education' feature since the info. is covered

In [18]:
train = train.drop('education')

In [19]:
df = crosstabPercentage(train, 'workclass', labelCol).orderBy('percentage-of->50K')
df = df.orderBy('percentage-of->50K').withColumn('percentage-of->50K', 
                                    format_number(df['percentage-of->50K'], 2))
df.show()

+-----+----+----------------+------------------+
|<=50K|>50K|  workclassClass|percentage-of->50K|
+-----+----+----------------+------------------+
|    7|   0|    Never-worked|              0.00|
|   14|   0|     Without-pay|              0.00|
| 1645| 191|               ?|             10.40|
|17733|4963|         Private|             21.87|
|  945| 353|       State-gov|             27.20|
| 1817| 724|Self-emp-not-inc|             28.49|
| 1476| 617|       Local-gov|             29.48|
|  589| 371|     Federal-gov|             38.65|
|  494| 622|    Self-emp-inc|             55.73|
+-----+----+----------------+------------------+



In [20]:
df = crosstabPercentage(train, 'hours-per-week', labelCol).orderBy('percentage-of->50K')
df = df.orderBy('percentage-of->50K').withColumn('percentage-of->50K', 
                                    format_number(df['percentage-of->50K'], 2))
df.show()

+-----+----+-------------------+------------------+
|<=50K|>50K|hours-per-weekClass|percentage-of->50K|
+-----+----+-------------------+------------------+
|    1|   0|                 94|              0.00|
|    1|   0|                 92|              0.00|
|   14|   0|                 19|              0.00|
|   11|   0|                 11|              0.00|
|   21|   0|                 23|              0.00|
|    5|   0|                 31|              0.00|
|    1|   0|                 74|              0.00|
|    6|   0|                 77|              0.00|
|    3|   0|                 81|              0.00|
|    1|   0|                 82|              0.00|
|    2|   0|                 86|              0.00|
|    1|   0|                 87|              0.00|
|    2|   0|                 88|              0.00|
|    3|   0|                 91|              0.00|
|   38|   1|                  3|              2.56|
|   29|   1|                 27|              3.33|
|   28|   1|

In [21]:
_, indexedTypes, indexedDf = autoIndexer(train, labelCol)
# The null hypothesis is that the occurrence of the outcomes is statistically independent.
# In general, small p-values (1% to 5%) would cause you to reject the null hypothesis. 
# This very large p-value (92.65%) means that the null hypothesis should not be rejected.
testResult = ChiSquareTest.test(indexedDf, 'assembled', 'classIndexed')
r = testResult.head()
print("pValues: " + str(r.pValues))
print("degreesOfFreedom: " + str(r.degreesOfFreedom))
print("statistics: " + str(r.statistics))


pValues: [0.0,6.06621672894e-08,2.35683250693e-09,0.0,0.0073969884446,0.0,0.0,0.0350527043814,0.0,0.0,0.0,0.0,0.0,1.65423230669e-14,0.0,0.0232540498908,0.0,0.0,1.91608792943e-05,0.0,0.0,0.0,0.000106491974853,0.0,0.0,3.53831313693e-06,3.89777829346e-07,2.10630402009e-11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0571069627625,2.18791128326e-07,0.0,4.9695980664e-10,0.0,0.583599768678,0.0263766737841,0.0274947609427,0.035664584289,0.000697135482754,0.000169808165581,0.000192370213887,0.609920556916,0.0398097636079,0.0133898002533,0.392700772659,0.600053722635,0.0419896247007,3.21813000361e-05,0.00144941904399,0.000281102267653,0.0070067904419,0.459306125248,0.000198893471588,0.0114124246819,0.0199611745997,0.00636173094476,0.0589039315422,0.0130248843455,0.0216336827864,0.658726919143,0.0292840513559,0.225228126282,0.709716719993,0.535731237288,0.166902075353,0.193152005373,0.461796906847,0.197990667677,0.209235051547,0.0350527043814,0.166892320718,0.932513756806,0.940634568087]
degreesOfFreedom: [1, 

In [22]:
from pyspark.ml.clustering import KMeans

kmeans = KMeans(k=2, featuresCol='assembled')
model = kmeans.fit(indexedDf)
indexedDf = model.transform(indexedDf)

In [23]:
indexedDf.select('prediction', 'classIndexed').show()

+----------+------------+
|prediction|classIndexed|
+----------+------------+
|         0|         0.0|
|         1|         0.0|
|         0|         0.0|
|         1|         0.0|
|         1|         0.0|
|         1|         0.0|
|         0|         0.0|
|         1|         1.0|
|         0|         1.0|
|         1|         1.0|
|         1|         1.0|
|         1|         1.0|
|         0|         0.0|
|         0|         0.0|
|         1|         1.0|
|         1|         0.0|
|         0|         0.0|
|         0|         0.0|
|         1|         0.0|
|         0|         1.0|
+----------+------------+
only showing top 20 rows



# Classification

In [24]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

stringTypes = [dtype[0] for dtype in train.dtypes if dtype[1] == 'string']
indexedTypes = [stringType+'Indexed' for stringType in stringTypes]

indexers = [StringIndexer(inputCol=stringType, outputCol=stringType+'Indexed', handleInvalid='skip') \
            for stringType in stringTypes]

In [25]:
from pyspark.ml.feature import OneHotEncoderEstimator, VectorAssembler
from pyspark.ml.classification import GBTClassifier

oheTypes = [indexedType+'OneHotEncoded' for indexedType in indexedTypes]
ohe = OneHotEncoderEstimator(inputCols=indexedTypes, outputCols=oheTypes)

# Fix columns
oheTypes.remove('classIndexedOneHotEncoded')
cols = train.columns[:]
for oheType in oheTypes:
    cols.append(oheType)
for stringType in stringTypes:
    cols.remove(stringType)

cols.remove('index')

assembler = VectorAssembler(inputCols=cols, outputCol='assembled')
classifier = GBTClassifier(featuresCol='assembled', labelCol='classIndexed')
pipeline = Pipeline(stages=[*indexers, ohe, assembler, classifier])
model = pipeline.fit(train)
train = model.transform(train)
train

DataFrame[age: int, workclass: string, fnlwgt: int, education-num: int, marital-status: string, occupation: string, relationship: string, race: string, sex: string, capital-gain: int, capital-loss: int, hours-per-week: int, native-country: string, class: string, index: bigint, workclassIndexed: double, marital-statusIndexed: double, occupationIndexed: double, relationshipIndexed: double, raceIndexed: double, sexIndexed: double, native-countryIndexed: double, classIndexed: double, workclassIndexedOneHotEncoded: vector, raceIndexedOneHotEncoded: vector, occupationIndexedOneHotEncoded: vector, relationshipIndexedOneHotEncoded: vector, native-countryIndexedOneHotEncoded: vector, marital-statusIndexedOneHotEncoded: vector, sexIndexedOneHotEncoded: vector, classIndexedOneHotEncoded: vector, assembled: vector, rawPrediction: vector, probability: vector, prediction: double]

Since we have class imbalance problem, that's why we will use area under ROC curve as metric.

In [26]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(labelCol='classIndexed', metricName='areaUnderROC')
metric = evaluator.evaluate(train)
metric

0.9187387194685324

In [27]:
classifier = model.stages[-1]
ohe = model.stages[-3]
ohe

OneHotEncoderEstimator_40df83385e3f8afb8b61

In [28]:
headers = ["age", "workclass", "fnlwgt", "education", "education-num",
           "marital-status", "occupation", "relationship", "race", "sex",
           "capital-gain", "capital-loss", "hours-per-week", "native-country",
           "class"]

test = spark.read.csv('./adult.test.txt',
                      inferSchema='true', 
                      ignoreLeadingWhiteSpace='true',
                      ignoreTrailingWhiteSpace='true').toDF(*headers)
test.select('class').show()

+------+
| class|
+------+
|<=50K.|
|<=50K.|
| >50K.|
| >50K.|
|<=50K.|
|<=50K.|
|<=50K.|
| >50K.|
|<=50K.|
|<=50K.|
| >50K.|
|<=50K.|
|<=50K.|
|<=50K.|
| >50K.|
| >50K.|
|<=50K.|
|<=50K.|
|<=50K.|
| >50K.|
+------+
only showing top 20 rows



We can see the class labels in the test dataset are different than in train - '>50K' and '>50K.'. So we have to remove the extrac dot from the class label, before evaluating.

In [29]:
from pyspark.sql.types import StringType
stripDot = udf(lambda s: s[:-1], StringType())

test = test.withColumn('classTrailed', stripDot(test['class'])).drop('class').withColumnRenamed('classTrailed', 'class')
test.select('class').show()

+-----+
|class|
+-----+
|<=50K|
|<=50K|
| >50K|
| >50K|
|<=50K|
|<=50K|
|<=50K|
| >50K|
|<=50K|
|<=50K|
| >50K|
|<=50K|
|<=50K|
|<=50K|
| >50K|
| >50K|
|<=50K|
|<=50K|
|<=50K|
| >50K|
+-----+
only showing top 20 rows



In [30]:
test = model.transform(test)
metric = evaluator.evaluate(test)
metric


0.9104077343632521